# NFL Play-by-Play Data Analysis

### Assist from https://towardsdatascience.com/analyzing-and-plotting-nfl-data-with-nflfastpy-and-plotly-a170a09cad6

In [1]:
import nfl_data_py as nfl
import pandas as pd
import plotly.graph_objects as go

In [2]:
df_2021 = nfl.import_pbp_data([2021])

2021 done.
Downcasting floats.


In [3]:
cols = list(nfl.see_pbp_cols())

In [4]:
cols

['play_id',
 'game_id',
 'old_game_id',
 'home_team',
 'away_team',
 'season_type',
 'week',
 'posteam',
 'posteam_type',
 'defteam',
 'side_of_field',
 'yardline_100',
 'game_date',
 'quarter_seconds_remaining',
 'half_seconds_remaining',
 'game_seconds_remaining',
 'game_half',
 'quarter_end',
 'drive',
 'sp',
 'qtr',
 'down',
 'goal_to_go',
 'time',
 'yrdln',
 'ydstogo',
 'ydsnet',
 'desc',
 'play_type',
 'yards_gained',
 'shotgun',
 'no_huddle',
 'qb_dropback',
 'qb_kneel',
 'qb_spike',
 'qb_scramble',
 'pass_length',
 'pass_location',
 'air_yards',
 'yards_after_catch',
 'run_location',
 'run_gap',
 'field_goal_result',
 'kick_distance',
 'extra_point_result',
 'two_point_conv_result',
 'home_timeouts_remaining',
 'away_timeouts_remaining',
 'timeout',
 'timeout_team',
 'td_team',
 'td_player_name',
 'td_player_id',
 'posteam_timeouts_remaining',
 'defteam_timeouts_remaining',
 'total_home_score',
 'total_away_score',
 'posteam_score',
 'defteam_score',
 'score_differential',
 'po

In [5]:
# import rosters for future join
rosters = nfl.import_rosters([2021])

In [6]:
# import teams (with colors) for future join
teams = nfl.import_team_desc()

In [7]:
# filter to regular season
df_2021 = df_2021[df_2021["season_type"] == "REG"]

# remove two point attempts
df_2021 = df_2021[df_2021["two_point_attempt"] == False]

# filter to pass plays
df_2021 = df_2021[df_2021["play_type"] == "pass"]

In [8]:
df_2021.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
3,76.0,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,TEN,home,ARI,...,0.0,1.0,0.032412,1.165133,5.803177,4.0,0.896654,0.125098,0.697346,30.265415
4,100.0,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,TEN,home,ARI,...,0.0,1.0,-1.532898,0.256036,4.147637,2.0,0.965009,0.965009,0.978253,2.174652
6,152.0,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,ARI,away,TEN,...,1.0,1.0,2.692890,0.567838,7.420427,4.0,1.000000,1.000000,0.458989,54.101131
8,218.0,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,ARI,away,TEN,...,1.0,1.0,-0.511090,1.036891,10.339405,9.0,0.478471,0.079696,0.684949,31.505138
9,253.0,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,ARI,away,TEN,...,1.0,1.0,2.182015,0.517965,3.045047,1.0,1.000000,0.998799,0.775463,22.453720


In [9]:
# join with the roster table to get player names
df_2021 = df_2021.merge(rosters[["player_name", "player_id"]], left_on="passer_player_id", right_on="player_id")

In [10]:
df_2021["player_name"].unique()

array(['Ryan Tannehill', 'Kyler Murray', 'Matthias Farley', 'Derek Carr',
       'Lamar Jackson', 'Andy Dalton', 'Justin Fields',
       'Matthew Stafford', 'Baker Mayfield', 'Patrick Mahomes',
       'Tom Brady', 'Dak Prescott', 'Daniel Jones', 'Teddy Bridgewater',
       'Jameis Winston', 'Aaron Rodgers', 'Taysom Hill', 'Jordan Love',
       'Tyrod Taylor', 'Trevor Lawrence', 'Justin Herbert',
       'Ryan Fitzpatrick', 'Taylor Heinicke', 'Mac Jones',
       'Tua Tagovailoa', 'Kirk Cousins', 'Joe Burrow', 'Justin Jefferson',
       'Zach Wilson', 'Sam Darnold', 'Matt Ryan', 'Jalen Hurts',
       'Josh Allen', 'Ben Roethlisberger', 'Carson Wentz',
       'Russell Wilson', 'Jared Goff', 'Jimmy Garoppolo', 'Trey Lance',
       'Josh Rosen', 'Jacoby Brissett', 'Davis Mills', 'Jacob Eason',
       'Greg Ward', "D'Andre Swift", 'Mitchell Trubisky', 'Drew Lock',
       'Ty Long', 'Jakobi Meyers', 'Geno Smith', 'Blaine Gabbert',
       'Kadarius Toney', 'Mike Glennon', 'Cedrick Wilson',
    

In [11]:
# join with team table to get team colors
df_2021 = df_2021.merge(teams[["team_abbr", "team_color"]], left_on="posteam", right_on="team_abbr")

In [12]:
# get total yards and touchdowns by week
df_agg = (
    df_2021.groupby(["player_name", "team_abbr", "team_color", "week"], as_index=False)
    .agg({"passing_yards": "sum", "pass_touchdown": "sum"})
)

In [13]:
# look at Josh Allen only
df_agg[df_agg["player_name"] == "Josh Allen"]

,player_name,team_abbr,team_color,week,passing_yards,pass_touchdown
299,Josh Allen,BUF,#00338D,1,270.0,1.0
300,Josh Allen,BUF,#00338D,2,179.0,2.0
301,Josh Allen,BUF,#00338D,3,358.0,4.0
302,Josh Allen,BUF,#00338D,4,248.0,2.0
303,Josh Allen,BUF,#00338D,5,315.0,3.0
304,Josh Allen,BUF,#00338D,6,353.0,3.0
305,Josh Allen,BUF,#00338D,8,249.0,2.0
306,Josh Allen,BUF,#00338D,9,264.0,0.0
307,Josh Allen,BUF,#00338D,10,366.0,2.0
308,Josh Allen,BUF,#00338D,11,209.0,2.0


In [16]:
fig = go.Figure()
for name, values in df_agg.groupby("player_name"):
    if values["passing_yards"].sum() > 1000:
        fig.add_trace(
            go.Scatter(
                x=values["week"], 
                y=values["passing_yards"].cumsum(), 
                name=name, 
                mode="markers+lines", 
                line_color=values.iloc[0].team_color,
                hovertemplate=f"<b>{name}</b><br>%{{y}} yds through week %{{x}}<extra></extra>"
            )
        )
    
fig.update_layout(
    font_family="Averta, sans-serif",
    hoverlabel_font_family="Averta, sans-serif",
    xaxis_title_text="Week",
    xaxis_title_font_size=18,
    xaxis_tickfont_size=16,
    yaxis_title_text="Passing Yards",
    yaxis_title_font_size=18,
    yaxis_tickfont_size=16,
    hoverlabel_font_size=16,
    legend_font_size=16,
    height=1000,
    width=1000
)
    
fig.show()

In [15]:
fig = go.Figure()
for name, values in df_agg.groupby("player_name"):
    if values["pass_touchdown"].sum() > 5:
        fig.add_trace(
            go.Scatter(
                x=values["week"], 
                y=values["pass_touchdown"].cumsum(), 
                name=name, 
                mode="markers+lines", 
                line_color=values.iloc[0].team_color,
                hovertemplate=f"<b>{name}</b><br>%{{y}} pass TDs through week %{{x}}<extra></extra>"
            )
        )
    
fig.update_layout(
    font_family="Averta, sans-serif",
    hoverlabel_font_family="Averta, sans-serif",
    xaxis_title_text="Week",
    xaxis_title_font_size=18,
    xaxis_tickfont_size=16,
    yaxis_title_text="Passing TDs",
    yaxis_title_font_size=18,
    yaxis_tickfont_size=16,
    hoverlabel_font_size=16,
    legend_font_size=16,
    height=1000,
    width=1000
)
    
fig.show()